In [ ]:
import numpy as np
import sklearn.manifold as skmfld
import sklearn.datasets as skdata
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Blobs

In [ ]:
def make_blob(center, radius, num_samples=100):
    """
    Args:
        center: length 2 list specifying x and y coords
        radius: number specifying variance in both coords
        num_samples: integer specifying number of samples, default=100
    Returns:
        np.array with shape (num_samples, 2); centered at center with stddev=radius
    """
    x = np.random.normal(center[0], radius, num_samples)
    y = np.random.normal(center[1], radius, num_samples)
    return np.c_[x, y]

def make_labeled_blobs(centers, radii):
    """
    Args:
        centers: list of centers (list of pairs of numbers)
        radii: list of radii (list of numbers)
    Returns:
        np.array of all the blobs, with a "label" column
    """
    label = 0
    arr_to_return = None
    for c, r in zip(centers, radii):
        this_cluster = make_blob(c, r)
        #exercise
        #label_col = #column of correct size with value label everywhere
        label_col = label * np.ones(this_cluster.shape[0])
        this_cluster_labeled = np.c_[this_cluster, label_col]
        if arr_to_return is None:
            arr_to_return = this_cluster_labeled
        else:
            arr_to_return = np.r_[arr_to_return, this_cluster_labeled]
        label += 1
    return arr_to_return

In [ ]:
easyblobs = make_labeled_blobs([[0,0],[5,10],[10,0]], [1,1,1])
print easyblobs.shape
plt.scatter(easyblobs[:,0], easyblobs[:,1], c=easyblobs[:,2])

In [ ]:
asymblobs = make_labeled_blobs([[0,0],[5,10],[40,5]], [1,1,1])
print asymblobs.shape
plt.scatter(asymblobs[:,0], asymblobs[:,1], c=asymblobs[:,2])

In [ ]:
blobs_var_density = make_labeled_blobs([[0,0],[5,10],[40,5]], [1,2,.5])
print blobs_var_density.shape
plt.scatter(blobs_var_density[:,0], blobs_var_density[:,1], c=blobs_var_density[:,2])

In [ ]:
hardblobs = make_labeled_blobs([[0,0],[5,5],[10,2]], [1,1,2])
print hardblobs.shape
plt.scatter(hardblobs[:,0], hardblobs[:,1], c=hardblobs[:,2])

# Swiss Rolls

In [ ]:
swiss_rolls = [skdata.make_swiss_roll(n_samples=n) for n in [100, 500, 1000, 5000]]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
Axes3D
# fix sub-plots
fig = plt.figure(figsize=(20, 4))
for ind, sr in enumerate(swiss_rolls):
    roll, coord = sr
    ax = fig.add_subplot(1,len(swiss_rolls),ind+1, projection='3d')
    ax.scatter(roll[:,0], roll[:,1], roll[:,2], c=coord, cmap=plt.cm.viridis)

# PCA

In [ ]:
import sklearn.decomposition as decomp
def try_pca(data, color):
    fig = plt.figure(figsize=(10,4))
    pca = decomp.PCA()
    data_red = pca.fit_transform(data)
    plt.scatter(data_red[:,0], data_red[:,1], c=color, cmap=plt.cm.viridis)

In [ ]:
# PCA on the blobs is the identity
try_pca(hardblobs, hardblobs[:,2])

In [ ]:
try_pca(swiss_rolls[0][0], swiss_rolls[0][1])

In [ ]:
try_pca(swiss_rolls[3][0], swiss_rolls[3][1])

## Exercises:
1. PCA actually does all right on these rolls. Can you change the rolls to confuse it?
2. Make the sandwich example (two filled ellipses separated by a small distance orthogonal to their axes). Which algorithms will perform well on this example? Test it.

# MDS

In [ ]:
def try_mds(data, color):
    fig = plt.figure(figsize=(10,4))
    mds = skmfld.MDS(n_components=2, n_jobs=-1, verbose=1)
    data_red = mds.fit_transform(data)
    plt.scatter(data_red[:,0], data_red[:,1], c=color, cmap=plt.cm.viridis)

In [ ]:
try_mds(hardblobs, color=hardblobs[:,2])

In [ ]:
try_mds(swiss_rolls[0][0], color=swiss_rolls[0][1])

In [ ]:
try_mds(swiss_rolls[1][0], color=swiss_rolls[1][1])

In [ ]:
# Warning -- pretty slow
try_mds(swiss_rolls[2][0], color=swiss_rolls[2][1])

In [ ]:
# Warning -- slow
# try_mds(swiss_rolls[3][0], color=swiss_rolls[3][1])

# Locally Linear Embedding
https://www.cs.nyu.edu/~roweis/lle/papers/lleintro.pdf

Exercises: 
1. Find the transition points between results.
2. Explain the results at low parameter choices.

In [ ]:
def try_lle(data, color, neighbor_choices=(1,10,50,100,200)):
    fig = plt.figure(figsize=(20, 4))
    for ind, n_neighbors in enumerate(neighbor_choices):
        lle = skmfld.LocallyLinearEmbedding(n_neighbors, n_components=2, n_jobs=-1, eigen_solver='dense')
        data_red = lle.fit_transform(data)
        ax = fig.add_subplot(1,5,ind + 1)
        ax.scatter(data_red[:,0], data_red[:,1], c=color, cmap=plt.cm.viridis)

In [ ]:
try_lle(easyblobs, easyblobs[:,2])

In [ ]:
try_lle(asymblobs, asymblobs[:,2])

In [ ]:
try_lle(blobs_var_density, blobs_var_density[:,2])

In [ ]:
try_lle(hardblobs, hardblobs[:,2])

In [ ]:
try_lle(swiss_rolls[0][0], color=swiss_rolls[0][1], neighbor_choices=(1,10,20,50,99))

In [ ]:
try_lle(swiss_rolls[1][0], color=swiss_rolls[1][1])

In [ ]:
# Warning -- pretty slow
# try_lle(swiss_rolls[2][0], color=swiss_rolls[2][1])

In [ ]:
# Warning -- slow
# try_lle(swiss_rolls[3][0], color=swiss_rolls[3][1])

# Isomap
http://web.mit.edu/cocosci/Papers/sci_reprint.pdf

Exercises: 
1. Find the transition points between results. 
2. Explain the results at low parameter choices.
3. Why do some of the blobs "blow up" before others in the last two blob examples?

In [ ]:
def try_isomap(data, color, neighbor_choices=(1,10,50,100,200)):
    fig = plt.figure(figsize=(20, 4))
    for ind, n_neighbors in enumerate(neighbor_choices):
        isomap = skmfld.Isomap(n_neighbors, n_components=2, n_jobs=-1)
        data_red = isomap.fit_transform(data)
        ax = fig.add_subplot(1,5,ind + 1)
        ax.scatter(data_red[:,0], data_red[:,1], c=color, cmap=plt.cm.viridis)

In [ ]:
# Exercise: Find the transition point. Justify it.
try_isomap(easyblobs, easyblobs[:,2])

In [ ]:
try_isomap(asymblobs, asymblobs[:,2])

In [ ]:
try_isomap(blobs_var_density, blobs_var_density[:,2])

In [ ]:
# Exercise: explain why the clusters "blow up" in the order they do.
try_isomap(hardblobs, hardblobs[:,2])

In [ ]:
# Exercise: find a parameter choice that actually unrolls the swiss roll.
try_isomap(swiss_rolls[0][0], swiss_rolls[0][1], neighbor_choices=(1,10,50,80,99))

In [ ]:
try_isomap(swiss_rolls[1][0], swiss_rolls[1][1], neighbor_choices=(1,10,50,100,200))

In [ ]:
try_isomap(swiss_rolls[2][0], swiss_rolls[2][1], neighbor_choices=(1,10,50,200,500))

In [ ]:
# Warning -- this is slow
# try_isomap(swiss_rolls[3][0], swiss_rolls[3][1], neighbor_choices=(1,10,100,200,500))

# Another algorithm, just for fun: t-SNE
https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf
but see also https://distill.pub/2016/misread-tsne/, which does not accomplish its title.

# Spheres

In [ ]:
def make_sphere(dim=4, num_points=500):
    array = np.random.normal(0,1,[num_points, dim+1])
    print array.shape
    presph = pd.DataFrame(array)
    print presph.shape
    row_norms = presph.apply(np.linalg.norm, axis=1)
    print row_norms.shape
    sph = presph.apply(lambda x: x/row_norms, axis=0)
    print sph.shape
    return sph

In [ ]:
sph2 = make_sphere(2)
# sph3 = make_sphere(3)
# sph4 = make_sphere()

In [ ]:
for x in range(3):
    try_pca(sph2, sph2[x])

In [ ]:
for x in range(3):
    try_isomap(sph2, sph2[x])

In [ ]:
for x in range(3):
    try_lle(sph2, sph2[x])

# Persistent homology references
http://www.ams.org/journals/bull/2009-46-02/S0273-0979-09-01249-X/S0273-0979-09-01249-X.pdf
https://www.math.upenn.edu/~ghrist/preprints/barcodes.pdf